# DiffractionOnly_v1 Overlay Test

Este notebook carga el overlay `DiffractionOnly_v1` y compara la salida del kernel con una implementación en software.
Asegúrate de colocar `DiffractionOnly_v1.bit` y `DiffractionOnly_v1.hwh` en esta carpeta antes de ejecutar.

In [ ]:
from pynq import Overlay, allocate
import numpy as np
import matplotlib.pyplot as plt
import time
from deep_tissue_imaging.elementos.lasers import fuente_microscopia_1 as laser, campo_tem00
from deep_tissue_imaging.elementos.tejidos import cerebro_emb_pez_cebra as tejido
from deep_tissue_imaging.propagators.step_operators import adi_x, adi_y

def plot_magnitude(field, title):
    plt.figure()
    plt.imshow(np.abs(field), cmap='viridis')
    plt.colorbar()
    plt.title(title)
    plt.show()


In [ ]:
# Cargar overlay
ol = Overlay('DiffractionOnly_v1.bit')
print('IPs disponibles:', list(ol.ip_dict.keys()))
ip_name = [k for k in ol.ip_dict if 'diffraction' in k][0]
ip = getattr(ol, ip_name)


In [ ]:
# Configurar dominio y campo inicial
Lz = np.float32(361e-6)
Nz = 361
dz = np.float32(Lz / Nz)
Lx = Ly = np.float32(45e-6)
Nx = Ny = 256
dx = np.float32(Lx / Nx)
dy = np.float32(Ly / Ny)
x = np.linspace(-Lx/2, Lx/2, Nx, dtype=np.float32)
y = np.linspace(-Ly/2, Ly/2, Ny, dtype=np.float32)
X, Y = np.meshgrid(x, y)
phi0 = campo_tem00(X, Y, laser.w0, laser.I_peak)

k0 = np.float32(2*np.pi / laser.wavelength)
k = np.float32(k0 * tejido.n_0)
eps = np.float32(1e-12)

plot_magnitude(phi0, 'Perfil inicial |phi0|')


In [ ]:
# Implementación en software usando ADI
def diffraction_sw(phi):
    tmp = adi_x(phi, Ny, eps, k, dz, dx)
    return adi_y(tmp, Nx, eps, k, dz, dy)


In [ ]:
# Ejecutar en software
t0 = time.time()
sw_out = diffraction_sw(phi0)
sw_time = time.time() - t0
print(f'Software time: {sw_time*1e3:.2f} ms')
plot_magnitude(sw_out, '|phi| después de SW')


In [ ]:
# Ejecutar en hardware
N = Nx * Ny
in_buffer = allocate(shape=(N,), dtype=np.complex64)
out_buffer = allocate(shape=(N,), dtype=np.complex64)
np.copyto(in_buffer, phi0.reshape(-1))

t0 = time.time()
ip.write(0x10, in_buffer.physical_address)
ip.write(0x18, out_buffer.physical_address)
ip.write(0x00, 1)  # ap_start
while (ip.read(0x00) & 0x2) == 0:
    pass
hw_time = time.time() - t0
hw_out = np.array(out_buffer).reshape(Nx, Ny)
print(f'Hardware time: {hw_time*1e3:.2f} ms')
plot_magnitude(hw_out, '|phi| después de HW')


In [ ]:
# Validación
diff = np.abs(sw_out - hw_out)
print('Max error:', diff.max())
print('HW faster by {:.2f}x'.format(sw_time / hw_time))
